In [1]:
import pandas as pd
from pathlib import Path

# Path to the CSV file (from inside the notebooks/ folder)
df=pd.read_csv("../data/raw_analyst_ratings.csv")


In [2]:

# Quick check of columns
df.head()


,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [3]:
# Ensure 'date' is string and strip spaces
df['date'] = df['date'].astype(str).str.strip()


In [4]:
# Parse all date formats; invalid ones become NaT
df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [5]:
# Drop timezone info if present
if pd.api.types.is_datetime64tz_dtype(df['date']):
    df['date'] = df['date'].dt.tz_localize(None)


C:\Users\filimon.hailemariam\AppData\Local\Temp\ipykernel_39920\4279842223.py:2: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df['date']):


In [6]:
start_date = df['date'].min()
end_date   = df['date'].max()

print("Start date:", start_date)
print("End date:  ", end_date)


Start date: 2011-04-27 21:01:48
End date:   2020-06-11 17:12:35


In [ ]:
from dateutil import parser

def parse_mixed_dates(x):
    try:
        # First: try normal parser (handles ISO, timezone, many formats)
        return parser.parse(x)
    except:
        try:
            # Second: explicitly parse US-style MM/DD/YYYY
            return pd.to_datetime(x, format="%m/%d/%Y %H:%M")
        except:
            return pd.NaT


In [4]:
# Unified date parsing
df['date'] = pd.to_datetime(
    df['date'],
    errors='coerce',
    infer_datetime_format=True,
    utc=False
)


C:\Users\filimon.hailemariam\AppData\Local\Temp\ipykernel_39704\2115666273.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(


In [6]:
# Normalize timezone-aware datetimes
df['date'] = df['date'].dt.tz_localize(None)


In [7]:
df['date'].head()


0   2020-06-05 10:30:54
1   2020-06-03 10:45:20
2   2020-05-26 04:30:07
3   2020-05-22 12:45:06
4   2020-05-22 11:38:59
Name: date, dtype: datetime64[ns]

In [8]:
print("Start date:", df['date'].min())
print("End date:", df['date'].max())


Start date: 2011-04-27 21:01:48
End date: 2020-06-11 17:12:35


In [ ]:
# Check if 'date' column exists before conversion
if 'date' in df.columns:
    # keep original strings for fallback attempts
    original_dates = df['date'].copy()

    # First pass: let pandas parse mixed timezone-aware and naive strings,
    # return timezone-aware timestamps in UTC; invalid parse -> NaT
    df['date'] = pd.to_datetime(original_dates, utc=True, errors='coerce', infer_datetime_format=True)

    # If any remain unparsed, try ISO8601 parsing as a fallback for those rows
    if df['date'].isna().any():
        mask = df['date'].isna()
        df.loc[mask, 'date'] = pd.to_datetime(original_dates[mask].astype(str),
                                              format='ISO8601',
                                              errors='coerce',
                                              utc=True)
        if df['date'].isna().any():
            n_bad = df['date'].isna().sum()
            print(f"Warning: {n_bad} unparsable date(s) converted to NaT.")
else:
    print("Error: 'date' column not found in DataFrame.")

# Confirm the type
df.dtypes


ValueError: time data "2020-05-22 00:00:00" doesn't match format "%Y-%m-%d %H:%M:%S%z", at position 10. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
start_date = df['date'].min()
end_date   = df['date'].max()

print(f"Data starts on: {start_date}")
print(f"Data ends on:   {end_date}")


In [ ]:
# Extract year from the date
df['year'] = df['date'].dt.year

# Count rows per year
year_counts = df.groupby('year').size().reset_index(name='n_rows')

year_counts
